## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.


In [46]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio
from typing import Dict, List, Any



In [2]:
load_dotenv(override=True)

True

## Step 1: Agent workflow

In [3]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [4]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [5]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Streamline Your SOC 2 Compliance with ComplAI

Hi [Recipient's Name],

I hope this message finds you well.

Navigating SOC 2 compliance can be a complex and daunting task. At ComplAI, we understand the challenges businesses face in maintaining compliance and preparing for audits. Our AI-powered SaaS tool is designed to simplify the compliance process, reduce time spent on administration, and ultimately enhance your security posture.

With ComplAI, you can:

- Automate routine compliance tasks, freeing up your team's time.
- Gain real-time insights into your compliance status.
- Prepare seamlessly for audits with organized documentation and reporting features.

I'd love to explore how ComplAI can be tailored to meet the needs of [Recipient's Company]. Are you available for a brief call next week?

Thank you for considering this opportunity. I look forward to the possibility of working together.

Best regards,

[Your Name]  
[Your Position]  
ComplAI  
[Your Phone Number]  
[You

In [6]:
result2 = Runner.run_streamed(sales_agent2, input="Write a humorous sales email. Resoponda em português.")
async for event in result2.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Assunto: O Super-Herói da Sua Conformidade: SOC2 do ComplAI!

Olá [Nome do Destinatário],

Você já sentiu que as auditorias são como aqueles filmes de terror? Você sabe, aqueles que vão ficando cada vez mais intensos, e você só deseja que acabe logo? 😱

Aqui na ComplAI, decidimos mudar o enredo! Apresento a você o nosso super-herói: o ComplAI! Com ele, suas dores de cabeça relacionadas a SOC2 e auditorias se transformarão em comédias de stand-up.

Imagine só: enquanto outras empresas ficam quebrando a cabeça e revivendo pesadelos, você poderá se sentar, relaxar e deixar que a nossa plataforma de IA cuide de toda a parte chata! 🎩✨

Se você está pronto para dizer adeus ao stress e olá à conformidade com um sorriso no rosto, vamos marcar um bate-papo! Prometo que não vou te fazer preencher um questionário de 10 páginas.

Aguardo seu sinal, seja ele um emoji, uma resposta, ou até mesmo um grito de socorro! 😂

Um abraço,

[Seu Nome]  
Seu Super-Agent do ComplAI  
🚀 [Seu Telefone] | 🌐 [Seu S

In [7]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


Subject: Streamline Your SOC 2 Compliance Process with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. I'm reaching out to introduce ComplAI, an innovative SaaS tool designed to simplify the SOC 2 compliance process for businesses like yours.

In today's regulatory landscape, ensuring compliance can be time-consuming and complex. Our AI-driven platform automates documentation, identifies gaps, and prepares your team for audits, helping you save both time and resources.

Here are a few ways ComplAI can benefit your organization:

- **Efficient Documentation Management**: Automate the creation of necessary compliance documentation to reduce manual effort.
- **Real-Time Gap Analysis**: Instantly identify compliance gaps and receive actionable insights for remediation.
- **Seamless Audit Preparation**: Simplify the audit process with organized data and tools to enhance collaboration among team members.

I would love the opportunity to discuss how ComplAI can specifical

In [8]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [9]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")


Best sales email:
Subject: Let’s Make Compliance Less of a “Comply-ance” Headache! 🎉

Hi [Recipient's Name],

I hope this email finds you knee-deep in success and avoiding audit-induced migraines! 🤯 

I’m [Your Name] from ComplAI, where we believe compliance shouldn’t feel like a dark, cryptic maze—especially when it comes to SOC 2. Let's face it: audits are like going to the dentist. You know you need to do it, but who actually enjoys the process?

That’s where we come in! Our AI-powered platform simplifies SOC 2 compliance faster than you can say, “Is it time for another coffee break?” ☕ With our tool, you’ll breeze through audits and make compliance a breeze instead of a chore.

Imagine a world where you spend less time assembling paperwork and more time celebrating your latest successes. Sounds dreamy, right? 

Would you be open to a quick chat to explore how we can turn your compliance chaos into clarity? I promise it will be more fun than a game of “Where’s Waldo?” but instead, w

Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [10]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)

In [11]:
sales_agent1

Agent(name='Professional Sales Agent', instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [31]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("williamlapa40@gmail.com")  # Change to your verified sender
    to_email = To("williamlapa@hotmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [ ]:
# @function_tool
# def send_email(body: str):
#     """ Send out an email with the given body to all sales prospects """
#     sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
#     from_email = Email("williamlapa40@gmail.com")  # Change to your verified sender
#     to_email = To("williamlapa@hotmail.com")  # Change to your recipient
#     content = Content("text/plain", body)
#     mail = Mail(from_email, to_email, "Sales email", content).get()

#     try:
#         response = sg.client.mail.send.post(request_body=mail)
#         # Imprima o status_code, body e headers para depuração
#         print(f"SendGrid Status Code: {response.status_code}")
#         print(f"SendGrid Response Body: {response.body}")
#         print(f"SendGrid Response Headers: {response.headers}")

#         if response.status_code >= 200 and response.status_code < 300:
#             return {"status": "success", "message": "Email request sent to SendGrid successfully."}
#         else:
#             return {"status": "error", "message": f"SendGrid API error: {response.status_code} - {response.body}"}
#     except Exception as e:
#         return {"status": "error", "message": f"An error occurred: {str(e)}"}

### This has automatically been converted into a tool, with the boilerplate json created

In [32]:
# Let's look at it
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000025A721C87C0>, strict_json_schema=True)

### And you can also convert an Agent into a tool

In [ ]:
# Transforma o agente numa tool
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
print(tool1)

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000025A7206B920>, strict_json_schema=True)


### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [34]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000025A721205E0>, strict_json_schema=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000025A721225C0>, strict_json_schema=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title'

## And now it's time for our Sales Manager - our planning agent

In [35]:
instructions ="You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales_agent tools once before choosing the best one. \
You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user. \
Elabore o texto em português do Brasil"


sales_manager = Agent(name="Sales manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'William'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

In [36]:
print(result)

RunResult:
- Last agent: Agent(name="Sales manager", ...)
- Final output (str):
    O email foi enviado com sucesso para o William. Se precisar de mais alguma coisa, é só avisar!
- 9 new item(s)
- 3 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Wait - you didn't get an email??</h2>
            <span style="color:#ff7800;">With much thanks to student Chris S. for describing his issue and fixes. 
            If you don't receive an email after running the prior cell, here are some things to check: <br/>
            First, check your Spam folder! Several students have missed that the emails arrived in Spam!<br/>Second, print(result) and see if you are receiving errors about SSL. 
            If you're receiving SSL errors, then please check out theses <a href="https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d">networking tips</a> and see the note in the next cell. Also look at the trace in OpenAI, and investigate on the SendGrid website, to hunt for clues. Let me know if I can help!
            </span>
        </td>
    </tr>
</table>

### And one more suggestion to send emails from student Oleksandr on Windows 11:

If you are getting certificate SSL errors, then:  
Run this in a terminal: `uv pip install --upgrade certifi`

Then run this code:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

Thank you Oleksandr!

## Remember to check the trace

https://platform.openai.com/traces

And then check your email!!


### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [37]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [38]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("williamlapa40@gmail.com")  # Change to your verified sender
    to_email = To("williamlapa@hotmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [39]:
tools = [subject_tool, html_tool, send_html_email]

In [40]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000025A721CA7A0>, strict_json_schema=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000025A721C9760>, strict_json_schema=True),
 FunctionTool(name='send_html_email', description='Send out an email with the given subject and HTML body to all sales prospects', params_json_

In [41]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it") # pass the handoff description explique?


### Now we have 3 tools and 1 handoff

In [42]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000025A721205E0>, strict_json_schema=True), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000025A721225C0>, strict_json_schema=True), FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 

In [44]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email. \
Elabore o texto em português do Brasil"


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to William Lapa from Alice"

with trace("Automated SDR em português"):
    result = await Runner.run(sales_manager, message)

### Remember to check the trace

https://platform.openai.com/traces

And then check your email!!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Can you identify the Agentic design patterns that were used here?<br/>
            What is the 1 line that changed this from being an Agentic "workflow" to "agent" under Anthropic's definition?<br/>
            Try adding in more tools and Agents! You could have tools that handle the mail merge to send to a list.<br/><br/>
            HARD CHALLENGE: research how you can have SendGrid call a Callback webhook when a user replies to an email,
            Then have the SDR respond to keep the conversation going! This may require some "vibe coding" 😂
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">This is immediately applicable to Sales Automation; but more generally this could be applied to  end-to-end automation of any business process through conversations and tools. Think of ways you could apply an Agent solution
            like this in your day job.
            </span>
        </td>
    </tr>
</table>

## Extra note:

Google has announced their Agent Development Kit (ADK) which is in early preview. It's still under development, so it's too early for us to cover it here. But it's interesting to note that it looks quite similar to OpenAI Agents SDK. To give you a preview, here's a peak at sample code from ADK:

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!

In [55]:
## Código extra para email merge
# --- Novas Ferramentas e Agentes para Mail Merge ---

# 1. Ferramenta para gerenciar a lista de prospects (simulada por enquanto)
@function_tool
def get_prospect_list() -> List[Dict[str, str]]:
    """
    Retorna uma lista de prospects de vendas com seus nomes e endereços de e-mail.
    """
    return [
        {"name": "William Gmail", "email": "williamlapa40@gmail.com"},
        {"name": "William Hotmail", "email": "williamlapa@hotmail.com"},
        {"name": "William RFB", "email": "william.lapa-filho@rfb.gov.br"}
    ]

# 2. Ferramenta para enviar e-mails personalizados (Mail Merge)
@function_tool
def send_personalized_email(subject: str, body_template: str, recipient_name: str, recipient_email: str) -> Dict[str, str]:
    """
    Envia um e-mail personalizado com o assunto e corpo fornecidos para um destinatário específico.
    O 'body_template' deve conter placeholders como [Recipient's Name] que serão substituídos.
    """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("williamlapa40@gmail.com")  # Altere para o seu remetente verificado

    # Substitui o placeholder no corpo do e-mail
    personalized_body = body_template.replace("[Recipient's Name]", recipient_name)
    personalized_body = personalized_body.replace("[Recipient Name]", recipient_name) # Adicionar mais placeholders
    personalized_body = personalized_body.replace("[First Name]", recipient_name.split()[0]) # Adicionar mais placeholders

    to_email = To(recipient_email)
    content = Content("text/plain", personalized_body)
    mail = Mail(from_email, to_email, subject, content).get()

    try:
        response = sg.client.mail.send.post(request_body=mail)
        if response.status_code >= 200 and response.status_code < 300:
            return {"status": "success", "message": f"Email enviado para {recipient_name} ({recipient_email})"}
        else:
            print(f"Erro SendGrid para {recipient_name}: {response.status_code} - {response.body}")
            return {"status": "error", "message": f"Erro SendGrid para {recipient_name}: {response.status_code}"}
    except Exception as e:
        print(f"Erro ao enviar email para {recipient_name}: {str(e)}")
        return {"status": "error", "message": f"Erro ao enviar email para {recipient_name}: {str(e)}"}

# 3. Agente para gerenciar o processo de Mail Merge
mail_merge_manager_instructions = """
Você é um gerente de mesclagem de correspondência.
Sua tarefa é orquestrar o envio de e-mails personalizados para uma lista de prospects.
1. Primeiro, você deve usar a ferramenta `get_prospect_list` para obter a lista de todos os prospects.
2. Para cada prospect na lista, você deve usar a ferramenta `send_personalized_email`.
   O `body_template` que você receberá será o corpo do e-mail principal a ser personalizado.
   O `subject` também será fornecido.
   Certifique-se de substituir os placeholders como '[Recipient\'s Name]' no `body_template` com o nome real do prospect.
   Elabore a saída e os retornos em português do Brasil.
"""

mail_merge_manager = Agent(
    name="Gerente de Mesclagem de Correspondência",
    instructions=mail_merge_manager_instructions,
    tools=[get_prospect_list, send_personalized_email],
    model="gpt-4o-mini"
)

# 4. Agente para lidar com o envio de e-mails HTML (atualizado para usar send_personalized_email se necessário)
@function_tool
def send_html_email(subject: str, html_body: str, recipient_email: str = "williamlapa@hotmail.com") -> Dict[str, str]:
    """ Envia um e-mail com o assunto e corpo HTML fornecidos para um único destinatário. """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("williamlapa40@gmail.com")  # Altere para o seu remetente verificado
    to_email = To(recipient_email)
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

# Emailer Agent (agora com a opção de handoff para mail_merge_manager)
emailer_agent_instructions = """
Você é um formatador e remetente de e-mails. Você recebe o corpo de um e-mail a ser enviado.
1. Você primeiro usa a ferramenta `subject_writer` para escrever um assunto para o e-mail.
2. Em seguida, você usa a ferramenta `html_converter` para converter o corpo para HTML.
3. Se a solicitação for para enviar para uma lista de prospects, você deve transferir o controle para o `Gerente de Mesclagem de Correspondência` passando o assunto e o corpo do e-mail formatado (texto, não HTML, para permitir a personalização).
4. Caso contrário, se for para um único destinatário, você usa a ferramenta `send_html_email` para enviar o e-mail com o assunto e o corpo HTML.
"""

emailer_agent = Agent(
    name="Gerente de E-mail",
    instructions=emailer_agent_instructions,
    tools=[subject_tool, html_tool, send_html_email], # send_html_email ainda é uma ferramenta para o caso de envio único
    handoffs=[mail_merge_manager], # Adiciona o Gerente de Mesclagem de Correspondência como handoff
    model="gpt-4o-mini",
    handoff_description="Converter um e-mail para HTML/personalizar e enviá-lo"
)

# Atualização do Sales Manager
sales_manager_instructions = """
Você é um gerente de vendas que trabalha para a ComplAI. Você usa as ferramentas dadas a você para gerar e-mails de vendas frios.
Você nunca gera e-mails de vendas sozinho; você sempre usa as ferramentas.
Você tenta todas as 3 ferramentas de agente de vendas pelo menos uma vez antes de escolher a melhor.
Você pode usar as ferramentas várias vezes se não estiver satisfeito com os resultados da primeira tentativa.
Você seleciona o melhor e-mail usando seu próprio julgamento sobre qual e-mail será mais eficaz.
Após escolher o e-mail, você transfere o controle para o agente `Gerente de E-mail` para formatar e enviar o e-mail.
Se a solicitação for para enviar para uma lista, especifique isso ao `Gerente de E-mail`.
Elabore o texto em português do Brasil.
"""

sales_manager = Agent(
    name="Gerente de Vendas",
    instructions=sales_manager_instructions,
    tools=[tool1, tool2, tool3], # Ferramentas de geração de e-mails
    handoffs=[emailer_agent], # Agora transfere para o Emailer Agent
    model="gpt-4o-mini"
)

In [56]:
# --- Teste do Novo Fluxo de Trabalho ---

async def run_scenario_single_email():
    print("\n--- Cenário: Enviar um único e-mail ---")
    message = "Envie um e-mail de vendas frio para William Lapa da remetente Alice."
    with trace("SDR Automatizado (E-mail Único)"):
        result = await Runner.run(sales_manager, message)
    print(result.final_output)

async def run_scenario_mail_merge():
    print("\n--- Cenário: Enviar e-mail para uma lista de prospects (Mail Merge) ---")
    # A mensagem precisa ser clara o suficiente para o Sales Manager entender que é uma lista
    message = "Envie um e-mail de vendas frio para uma lista de prospects, o e-mail deve ser sobre a ComplAI, da remetente Alice."
    with trace("SDR Automatizado (Mail Merge)"):
        result = await Runner.run(sales_manager, message)
    print(result.final_output)

# Executar os cenários
async def main():
    await run_scenario_single_email()
    await run_scenario_mail_merge()

# Descomente para executar o main()
await main()


--- Cenário: Enviar um único e-mail ---
Os e-mails foram enviados com sucesso para os três prospects:

1. **William Gmail**: (williamlapa40@gmail.com) - E-mail de vendas com assunto "Transforme sua Conformidade SOC2 com a ComplAI".
2. **William Hotmail**: (williamlapa@hotmail.com) - E-mail divertido com assunto "Vamos tornar a Compliance mais divertida? 🎉".
3. **William RFB**: (william.lapa-filho@rfb.gov.br) - E-mail sobre potencial colaboração em Compliance com o assunto "Potencial Colaboração em Compliance de SOC2".

Se precisar de mais alguma coisa, estarei à disposição!

--- Cenário: Enviar e-mail para uma lista de prospects (Mail Merge) ---
O e-mail foi preparado e transferido para o Gerente de Mesclagem de Correspondência. Ele irá cuidar do envio para a lista de prospects. Se precisar de mais alguma coisa, é só avisar!


In [58]:
print(result)

RunResult:
- Last agent: Agent(name="Email Manager", ...)
- Final output (str):
    O e-mail foi enviado com sucesso para o William Lapa! Se precisar de mais alguma coisa, estou à disposição.
- 16 new item(s)
- 5 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
